In [1]:
import keras
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Lambda
from keras.models import Model
from keras import backend as K
from keras import regularizers
from keras import layers
from keras.callbacks import TensorBoard
import numpy as np

Using TensorFlow backend.


In [2]:
# Code 5: Autoencoder, CNN, Varitional

In [3]:
input_img = Input(shape=(28, 28, 1)) 
encoding_dim = 32

encoded = Conv2D(16, (3,3), activation='relu', padding='same')(input_img)
encoded = MaxPooling2D((2, 2), padding='same')(encoded)
encoded = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
encoded = MaxPooling2D((2, 2), padding='same')(encoded)
encoded = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
encoded = MaxPooling2D((2, 2), padding='same')(encoded)
shape_before_flattening = K.int_shape(encoded)
encoded = layers.Flatten()(encoded)
encoded = layers.Dense(32, activation='relu')(encoded)

In [4]:
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),mean=0., stddev=1.)
    return z_mean + K.exp(z_log_var) * epsilon

In [5]:
# Variational Layer (Built into encoder model):
latent_dim = 2

z_mean = layers.Dense(latent_dim,name='mean')(encoded)
z_log_var = layers.Dense(latent_dim,name='var')(encoded)
z = layers.Lambda(sampling)([z_mean, z_log_var])
z = layers.Dense(np.prod(shape_before_flattening[1:]),activation='relu')(z)
output_encoded = layers.Reshape(shape_before_flattening[1:])(z)

encoder = Model(input_img,output_encoded,name='variational')

In [6]:
#encoder.summary()

In [7]:
encoded_input = Input(shape=(4,4,8))
decoded = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded_input)
decoded = UpSampling2D((2, 2))(decoded)
decoded = Conv2D(8, (3, 3), activation='relu', padding='same')(decoded)
decoded = UpSampling2D((2, 2))(decoded)
decoded = Conv2D(16, (3, 3), activation='relu')(decoded)
decoded = UpSampling2D((2, 2))(decoded)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(decoded)

decoder = Model(encoded_input,decoded)

In [8]:
#encoder.summary()
#decoder.summary()

In [17]:
# Variational Custom Loss Function:
def custom_loss(varlayer_mean,varlayer_var):
    def vae_loss(x, x_decoded_mean):
        xent_loss = K.binary_crossentropy(x, x_decoded_mean)
        kl_loss = - 0.5 * K.mean(1 + varlayer_var.output - K.square(varlayer_mean.output) - K.exp(varlayer_var.output), axis=-1)
        return xent_loss + kl_loss
    return vae_loss

In [18]:
# Combine and Compile:
autoencoder = Model(input_img,decoder(encoder(input_img)))
autoencoder.compile(optimizer='rmsprop', loss=custom_loss(autoencoder.get_layer('variational').get_layer('mean'),autoencoder.get_layer('variational').get_layer('var')))

In [19]:
# Import MNIST Data:
from keras.datasets import mnist
import numpy as np
(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.astype('float32') / 255.
x_test = x_test.reshape(x_test.shape + (1,))

In [ ]:
history = autoencoder.fit(x_train, x_train,
                          epochs=50,
                          batch_size=500,
                          shuffle=True,
                          validation_data=(x_test, x_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 1090s 18ms/step - loss: 0.2636 - val_loss: 0.2633
Epoch 2/50
60000/60000 [==============================] - 2798s 47ms/step - loss: 0.2633 - val_loss: 0.2634
Epoch 3/50
60000/60000 [==============================] - 1834s 31ms/step - loss: 0.2633 - val_loss: 0.2631
Epoch 4/50
17000/60000 [=======>......................] - ETA: 8:32:08 - loss: 0.2633

In [60]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
variational (Model)          (None, 4, 4, 8)           6548      
_________________________________________________________________
model_7 (Model)              (None, 28, 28, 1)         2481      
Total params: 9,029
Trainable params: 9,029
Non-trainable params: 0
_________________________________________________________________


In [ ]:
decoder.summary()